In [1]:
!mkdir tmp
!curl -fSL -o tmp/heart_2022_no_nans.csv https://raw.githubusercontent.com/AsherJingkongChen/heart-attack-prediction-course-project/main/data/heart_2022_no_nans.csv
%pip install numpy pandas tensorflow scikit-learn

# Read data
import numpy as np
import pandas as pd
df = pd.read_csv("tmp/heart_2022_no_nans.csv")
df

mkdir: tmp: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78.1M  100 78.1M    0     0  1324k      0  0:01:00  0:01:00 --:--:-- 1854k00:50 1243k01:02  0:00:11  0:00:51  999k:01:12  0:00:22  0:00:50  869k     0  0:01:08  0:00:42  0:00:26 1690k     0  1256k      0  0:01:03  0:00:49  0:00:14 1740k


,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,No,8.0,"6 or more, but not all",No,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3.0,15.0,Within past year (anytime less than 12 months ...,Yes,5.0,1 to 5,No,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246017,Virgin Islands,Male,Very good,0.0,0.0,Within past 2 years (1 year but less than 2 ye...,Yes,6.0,None of them,No,...,1.78,102.06,32.28,Yes,No,No,No,"Yes, received tetanus shot but not sure what type",No,No
246018,Virgin Islands,Female,Fair,0.0,7.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.93,90.72,24.34,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes
246019,Virgin Islands,Male,Good,0.0,15.0,Within past year (anytime less than 12 months ...,Yes,7.0,1 to 5,No,...,1.68,83.91,29.86,Yes,Yes,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,Yes
246020,Virgin Islands,Female,Excellent,2.0,2.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.70,83.01,28.66,No,Yes,Yes,No,"Yes, received tetanus shot but not sure what type",No,No


In [ ]:
# 切分自變數, 應變數
X = df.iloc[:, [i for i in range(df.shape[1]) if df.columns[i] != "HadHeartAttack"]] # 自變數
Y = df.loc[:, ["HadHeartAttack"]] # 應變數

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Use one-hot encoding
X_result = pd.DataFrame()
for i in range(X.shape[1]):
    if type(X.iloc[0,i]) == str:
        base_name = X.columns[i]
        this_column = pd.get_dummies(X.iloc[:, i])
        this_column = this_column.rename(columns = {n:"{}_{}".format(base_name, n) for n in this_column.columns})
        this_column = this_column.drop(this_column.columns[0], axis = 1)
    else:
        this_column = X.iloc[:, i]
    X_result = pd.concat([X_result, this_column], axis = 1)
X = X_result

# Label encoding
labelEncoder = LabelEncoder() # 建立物件
labelEncoder = labelEncoder.fit(Y) # 把類別資料數位化轉成 float64
Y = labelEncoder.transform(Y).astype("float64")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Data spliting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, random_state=0)

# Feature scaling
sc_X = StandardScaler().fit(X_train)
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Remove any logs from previous runs
!rm -rf ./logs/

# Create TensorBoard log directory
import os
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
# Start the TensorBoard (Ignore the errors)
%tensorboard --logdir logs --host=127.0.0.1 --port=6006 --load_fast=false

# Workaround for loading TensorBoard
from google.colab import output
output.serve_kernel_port_as_window(6006, path="")

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import metrics
# initializer = tf.keras.initializers.VarianceScaling(scale=2, mode='fan_in', distribution='truncated_normal')

# model = Sequential()
# model.add(Dense(input_dim = X_train.shape[1], units = 4000, kernel_initializer = 'rmsprop', activation = "relu"))
# model.add(Dense(units=10, kernel_initializer = 'rmsprop', activation = "relu"))
# model.add(Dense(units=1, kernel_initializer = 'rmsprop', activation = "sigmoid"))
# model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = [tf.keras.metrics.AUC()])
# model.fit(x = X_train, y = Y_train, validation_split = 0.2, batch_size = 5, epochs = 10, callbacks = [tensorboard_callback])
# Setup the HParams directory
hparams_dir = os.path.join(logdir, 'validation')

# Define Hyper-parameters  to
HP_NUM_FIRST_UNITS = hp.HParam('num_first_units', hp.Discrete([6000, 6937, 7800])) # num_units of second layer
HP_NUM_SECOND_UNITS = hp.HParam('num_second_units', hp.Discrete([3, 7, 14])) # num_units of second layer
HP_WEIGHT_INIT = hp.HParam('kernel_initializer', hp.Discrete(["random_normal", "glorot_uniform", "glorot_normal"])) # Different initializer
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['nadam', 'adam'])) # Different optimizer

HPARAMS = [HP_NUM_FIRST_UNITS, HP_NUM_SECOND_UNITS, HP_WEIGHT_INIT, HP_OPTIMIZER]

# Set the Standard of Evaluation
METRIC_AUC = "auc"

#Creating & configuring a log file writer
with tf.summary.create_file_writer(hparams_dir).as_default():
    hp.hparams_config(
        hparams=HPARAMS,
        metrics=[hp.Metric(METRIC_AUC, display_name='AUC')]
    )

# Count the total run
total_run = 1
for p in HPARAMS:
  total_run *= len(p.domain.values)

curr_run = 1
for num_first_units in HP_NUM_FIRST_UNITS.domain.values:
    for num_second_units in HP_NUM_SECOND_UNITS.domain.values:
        for weight_init in HP_WEIGHT_INIT.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_NUM_FIRST_UNITS: num_first_units,
                    HP_NUM_SECOND_UNITS: num_second_units,
                    HP_WEIGHT_INIT: weight_init,
                    HP_OPTIMIZER: optimizer
                }

                # Show somethings on Standard Output for progress
                run_name = "{} of {}".format(curr_run, total_run)
                print('--- Starting trial: {}'.format(run_name))
                print({h.name: hparams[h] for h in hparams})

                # Write information into log
                with tf.summary.create_file_writer(hparams_dir + "/" + run_name).as_default():
                    hp.hparams(hparams)

                    # Create & Train the model
                    theNodes = [hparams[HP_NUM_FIRST_UNITS], hparams[HP_NUM_SECOND_UNITS], 1]
                    model = Sequential()
                    model.add(Dense(input_dim = X_train.shape[1], units = theNodes[0], kernel_initializer = hparams[HP_WEIGHT_INIT], activation = "relu"))
                    model.add(Dense(units=theNodes[1], kernel_initializer = hparams[HP_WEIGHT_INIT], activation = "relu"))
                    model.add(Dense(units=theNodes[2], kernel_initializer = hparams[HP_WEIGHT_INIT], activation = "sigmoid"))
                    model.compile(optimizer = hparams[HP_OPTIMIZER], loss = "binary_crossentropy", metrics = [tf.keras.metrics.AUC()])
                    model.fit(x = X_train, y = Y_train, validation_split = 0.2, batch_size = 5, epochs = 5, callbacks = [tensorboard_callback, hp.KerasCallback(hparams_dir, hparams)])
                    # Evaluate the AUC & Write to log
                    test_loss, test_auc = model.evaluate(X_test, Y_test)
                    tf.summary.scalar(METRIC_AUC, test_auc, step=10)
                curr_run += 1

--- Starting trial: 1 of 54
{'num_first_units': 6000, 'num_second_units': 3, 'kernel_initializer': 'glorot_normal', 'optimizer': 'adam'}
Epoch 1/5
27555/27555 [==============================] - 201s 7ms/step - loss: 0.1640 - auc: 0.8484 - val_loss: 0.1513 - val_auc: 0.8709
Epoch 2/5
27555/27555 [==============================] - 187s 7ms/step - loss: 0.1507 - auc: 0.8731 - val_loss: 0.1619 - val_auc: 0.8679
Epoch 3/5
27555/27555 [==============================] - 189s 7ms/step - loss: 0.1490 - auc: 0.8773 - val_loss: 0.1518 - val_auc: 0.8783
Epoch 4/5
27555/27555 [==============================] - 182s 7ms/step - loss: 0.1480 - auc: 0.8796 - val_loss: 0.1501 - val_auc: 0.8761
Epoch 5/5
2307/2307 [==============================] - 7s 3ms/step - loss: 0.1481 - auc: 0.8802
--- Starting trial: 2 of 54
{'num_first_units': 6000, 'num_second_units': 3, 'kernel_initializer': 'glorot_normal', 'optimizer': 'nadam'}
Epoch 1/5
27555/27555 [==============================] - 195s 7ms/step - loss: 0.

In [ ]:
# Train
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import metrics
model = Sequential()
model.add(Dense(input_dim = X_train.shape[1], units = 6000, kernel_initializer = 'random_normal', activation = "relu"))
model.add(Dense(units=3, kernel_initializer = 'random_normal', activation = "relu"))
model.add(Dense(units=1, kernel_initializer = 'random_normal', activation = "sigmoid"))
model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = [tf.keras.metrics.AUC()])
model.fit(x = X_train, y = Y_train, validation_split = 0.2, batch_size = 5, epochs = 1, callbacks = [tensorboard_callback])

27555/27555 [==============================] - 186s 7ms/step - loss: 0.1552 - auc_2: 0.8626 - val_loss: 0.1509 - val_auc_2: 0.8729


In [ ]:
# Predict
Y_pred = model.predict(X_test)
test_loss, test_auc = model.evaluate(X_test, Y_test)
print("Test loss is: ", test_loss)
print("Test auc is: ", test_auc)

2307/2307 [==============================] - 6s 3ms/step - loss: 0.1484 - auc_2: 0.8785
Test loss is:  0.14842826128005981
Test auc is:  0.8785098195075989
